In [29]:
import tensorflow as tf

class NaiveDense:
  def __init__(self,input_size,output_size,activation):
    self.activation=activation

    w_shape=(input_size,output_size) #build array W
    w_initial_value=tf.random.uniform(w_shape,minval=0,maxval=1e-1) #array initial set to random(0 to 0.1)
    self.w=tf.Variable(w_initial_value) #build w Variable

    b_shape=(output_size,) #build b Vector
    b_inital_value=tf.zeros(b_shape) #make Vetor inital to 0
    self.b=tf.Variable(b_inital_value) #build b Variable

  def __call__(self,inputs): #build forward-propagation method
    return self.activation(tf.matmul(inputs,self.w)+self.b)

  @property #set weights only read can't rewrite
  def weights(self): #get Desen Weights
    return [self.w,self.b]

In [22]:
class NaiveSequential:
  def __init__(self,layers):
    self.layers = layers

  def __call__(self,inputs):
    x = inputs         #
    for layer in self.layers: # inputs data pass to all layers
      x = layer(x)      #
    return x

  @property
  def weights(self):
    weights = []
    for layer in self.layers:
      weights += layer.weights
    return weights

In [30]:
model = NaiveSequential([
    NaiveDense(input_size=28 * 28, output_size=512, activation=tf.nn.relu),
    NaiveDense(input_size=512, output_size=10, activation=tf.nn.softmax)
])
assert len(model.weights) == 4

In [31]:
import math

class BatchGenerator:
  def __init__(self,images,labels,batch_size=128):

    assert len(images) ==len(labels) #check images have correct labels
    self.index=0
    self.images=images
    self.labels=labels
    self.batch_size=batch_size
    self.num_batches=math.ceil(len(images)/batch_size) #account each batch

  def next(self):
    images=self.images[self.index:self.index+self.batch_size]
    labels=self.labels[self.index:self.index+self.batch_size]
    self.index+=self.batch_size
    return images,labels

In [32]:
def one_training_step(model,images_batch,labels_batch):
  with tf.GradientTape()as tape: #recode forward-propagation accountgraph
    predictions = model(images_batch) #input each batch images
    per_sample_losses=(tf.keras.losses.sparse_categorical_crossentropy(labels_batch,predictions)) #account each sample loss
    average_loss=tf.reduce_mean(per_sample_losses) #average this batch loss

  gradients=tape.gradient(average_loss,model.weights)

  update_weights(gradients,model.weights)
  return average_loss

learning_rate=1e-3

def update_weights(gradients,weights):
  for g,w,in zip(gradients,weights): #update each weights
    w.assign_sub(g*learning_rate)

In [33]:
def fit(model,images,labels,epochs,batch_size=128): #定義fit訓練迴圈

  for epoch_counter in range(epochs): #執行多少epoch次數
    print(f"Epoch {epoch_counter}")
    batch_generator=BatchGenerator(images,labels) #建立小批次訓練資料的產生器

    for batch_counter in range(batch_generator.num_batches):
      images_batch,labels_batch=batch_generator.next() #取出小批次的訓練資料

      loss=one_training_step(model,images_batch,labels_batch)
      if batch_counter % 100 ==0:
        print(f"loss batch {batch_counter}:{loss:.2f}")


In [34]:
from tensorflow.keras.datasets import mnist
(train_images,train_labels),(test_images,test_labels)=mnist.load_data()

train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype("float32") / 255

fit(model,train_images,train_labels,epochs=10,batch_size=128)

Epoch 0
loss batch 0:6.01
loss batch 100:2.21
loss batch 200:2.17
loss batch 300:2.04
loss batch 400:2.17
Epoch 1
loss batch 0:1.89
loss batch 100:1.86
loss batch 200:1.80
loss batch 300:1.66
loss batch 400:1.79
Epoch 2
loss batch 0:1.57
loss batch 100:1.56
loss batch 200:1.48
loss batch 300:1.39
loss batch 400:1.49
Epoch 3
loss batch 0:1.32
loss batch 100:1.33
loss batch 200:1.22
loss batch 300:1.18
loss batch 400:1.27
Epoch 4
loss batch 0:1.13
loss batch 100:1.15
loss batch 200:1.03
loss batch 300:1.02
loss batch 400:1.11
Epoch 5
loss batch 0:0.98
loss batch 100:1.01
loss batch 200:0.89
loss batch 300:0.91
loss batch 400:0.99
Epoch 6
loss batch 0:0.88
loss batch 100:0.91
loss batch 200:0.79
loss batch 300:0.82
loss batch 400:0.91
Epoch 7
loss batch 0:0.80
loss batch 100:0.82
loss batch 200:0.72
loss batch 300:0.75
loss batch 400:0.84
Epoch 8
loss batch 0:0.74
loss batch 100:0.76
loss batch 200:0.66
loss batch 300:0.70
loss batch 400:0.79
Epoch 9
loss batch 0:0.69
loss batch 100:0.70


In [38]:
import numpy as np
predictions=model(test_images)
predictions=predictions.numpy()

predictions_labels=np.argmax(predictions,axis=1)
matches = predictions_labels == test_labels
print(f"acc:{matches.mean():.2f}")

acc:0.82
